# AutoGen with MCP Workbench Tutorial

This notebook follows on from the **MCP Echo Server Tutorial** and shows how to use the `McpWorkbench` class to connect to an MCP server, list its tools, call them dynamically, override tool names/descriptions, and integrate directly into an AutoGen `AssistantAgent`.

## What is `McpWorkbench`?

The `McpWorkbench` is a higher-level interface for working with MCP servers. Instead of manually wrapping each MCP tool (like with `StreamableHttpMcpToolAdapter`), you connect to the entire server. The workbench can:

- **List available tools**
- **Call tools directly**
- **Override tool names and descriptions**
- **Integrate all tools into an agent automatically**

| Feature | MCP Tool Adapter | MCP Workbench |
| :--- | :--- | :--- |
| Connects to a single tool | ✅ | ❌ (connects to all tools) |
| Lists available tools | ❌ | ✅ |
| Calls tools directly | ✅ (wrapped only) | ✅ (any tool) |
| Supports tool overrides | ❌ | ✅ |
| Agent integration | Manual per tool | Automatic for all tools |

## Prerequisites
Make sure you have the `mcp_echo_tool.py` server running from the previous tutorial.

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai,mcp]>=0.7" "mcp>=1.0.0"

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

## 1. Connect to the MCP Echo Server with `McpWorkbench`

This example lists available tools and calls the first one.

In [2]:
import asyncio
from autogen_ext.tools.mcp import McpWorkbench, StreamableHttpServerParams

async def basic_workbench_demo():
    params = StreamableHttpServerParams(url="http://127.0.0.1:8000/mcp")
    
    async with McpWorkbench(server_params=params) as workbench:
        tools = await workbench.list_tools()
        print("Available tools:", tools)

        if tools:
            name = tools[0]["name"]
            print(f"Calling tool: {name}")
            result = await workbench.call_tool(name, {"text": "Hello from Workbench"})
            print("Result:", result)

await basic_workbench_demo()

Available tools: [{'name': 'echo', 'description': 'Echo the input text', 'parameters': {'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'additionalProperties': False}}]
Calling tool: echo
Result: type='ToolResult' name='echo' result=[TextResultContent(type='TextResultContent', content='Hello from Workbench')] is_error=False


## 2. Using Tool Overrides

We can change how tools appear to the agent without modifying the MCP server.

In [3]:
from autogen_core.tools import ToolOverride

async def workbench_with_overrides():
    overrides = {
        "echo": ToolOverride(
            name="repeat_message",
            description="Repeat back a message you send"
        )
    }
    params = StreamableHttpServerParams(url="http://127.0.0.1:8000/mcp")

    async with McpWorkbench(server_params=params, tool_overrides=overrides) as workbench:
        tools = await workbench.list_tools()
        print("Overridden tools:", tools)
        result = await workbench.call_tool("repeat_message", {"text": "Overridden tool call works!"})
        print("Result:", result)

await workbench_with_overrides()

Overridden tools: [{'name': 'repeat_message', 'description': 'Repeat back a message you send', 'parameters': {'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'additionalProperties': False}}]
Result: type='ToolResult' name='repeat_message' result=[TextResultContent(type='TextResultContent', content='Overridden tool call works!')] is_error=False


## 3. Connecting an Agent to the Workbench

When you pass the `McpWorkbench` to an `AssistantAgent`, it can automatically discover and use all tools from the MCP server.

In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

async def agent_with_workbench():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    params = StreamableHttpServerParams(url="http://127.0.0.1:8000/mcp")

    async with McpWorkbench(server_params=params) as mcp:
        agent = AssistantAgent(
            "echo_agent",
            model_client=model_client,
            workbench=mcp
        )
        await Console(agent.run_stream(task="Echo 'This came via the workbench!'"))

    await model_client.close()

await agent_with_workbench()

---------- TextMessage (user) ----------
Echo 'This came via the workbench!'
---------- ToolCallRequestEvent (echo_agent) ----------
[FunctionCall(id='call_VefXQnARZ9LsAHGkpc6Fpjsr', arguments='{"text":"This came via the workbench!"}', name='echo')]
---------- ToolCallExecutionEvent (echo_agent) ----------
[FunctionExecutionResult(content='This came via the workbench!', name='echo', call_id='call_VefXQnARZ9LsAHGkpc6Fpjsr', is_error=False)]
---------- ToolCallSummaryMessage (echo_agent) ----------
This came via the workbench!


## Summary

In this notebook you learned how to:

- Connect to an MCP server using `McpWorkbench`
- List and call tools dynamically
- Override tool names and descriptions
- Integrate the workbench directly into an AutoGen agent

Next, try connecting the workbench to other MCP servers like:
- **mcp-server-fetch** for fetching web content
- **GitHub MCP Server** for repository queries
- **Playwright MCP Server** for web automation

See the [api docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.mcp.html#autogen_ext.tools.mcp.McpWorkbench) for the above examples.

📚 Resources:
- [MCP Documentation](https://modelcontextprotocol.io)
- [AutoGen Documentation](https://microsoft.github.io/autogen)
- [Python MCP SDK](https://github.com/modelcontextprotocol/python-sdk)